GNM + RMSD vs xtal structure

In [2]:
#import 
import MDAnalysis as mda
from MDAnalysis.analysis import gnm
from MDAnalysis.analysis import rms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [3]:
# prepare universe(s)
unis = list()      # create list of universes
refs = list()    # create list of reference universes
ulabels = ['298K', '490K']                      # universe labels
rlabels = ['solv','Acetone','CypA','FAB']       # reference xtal structure labels
labells = ['298K','298K','298K','298K','490K','490K','490K','490K']
fig_labl = ['Ace_298_SPC_Ace','Ace_298_SPC_Cyp','Ace_298_SPC_FAB','Ace_490_SPC_Ace','Ace_490_SPC_Cyp','Ace_490_SPC_FAB']

#'convention=STRUCT_T_SOLV_RMSDvs'

# target universes
unis.append(mda.Universe("md_0_1_298.gro","md_0_1_298.xtc"))
unis.append(mda.Universe("md_0_1_490.gro","md_0_1_490.xtc"))

# reference universes for xtal RMSD comparison
#refs.append(mda.Universe("csa_solv_ions.gro"))     # define the reference universe
refs.append(mda.Universe("gro_rmsdstructs/csa_acetone_noh.gro"))    # def ref unis
refs.append(mda.Universe("gro_rmsdstructs/csa_cyp_noh.gro"))    # def ref unis
refs.append(mda.Universe("gro_rmsdstructs/csa_fab_noh.gro"))    # def ref unis
for r in refs:
    r.trajectory[-1] # set to last frame

In [5]:
# prepare GNM
nma = list()            # 
eigenvalues = list()    
#time = list()
nma_cb = list()
ev_cb = list()

# calpha atoms (backbone)
for i in list(range(len(unis))):    # for the number of plots to be made on the same graph,
    nma.append(gnm.GNMAnalysis(unis[i], select='name CA', cutoff=7.0))    # create,
    nma[i].run()            #  run the analyses
    eigenvalues.append([res[1] for res in nma[i].results])      # calculate eigenvalues
    #time.append([res[0] for res in nma[i].results])             # calculate times

# sidechain atoms
for i in list(range(len(unis))):    # for the number of plots to be made on the same graph,
    nma_cb.append(gnm.GNMAnalysis(unis[i], select='name CB', cutoff=7.0))    # create,
    nma_cb[i].run()            #  run the analyses
    ev_cb.append([res[1] for res in nma_cb[i].results])      # calculate eigenvalues
    #time.append([res[0] for res in nma[i].results])             # calculate times

In [6]:
time = list(range(5001))       # fix time differences (scale by 10)

In [10]:
#sel = 'not backbone and not type H and resnum 2:' 
u = mda.Universe("md_0_1_490.gro","md_0_1_490.xtc")
sel = 'resnum 1 or resnum 2 or resnum 3 and name CB'
print('Target uni:\t'+str(len(u.select_atoms(sel).positions)))
for r in refs:
    print('Reference uni:\t'+str(len(r.select_atoms(sel).positions)))

Target uni:	3
Reference uni:	3
Reference uni:	3
Reference uni:	3


In [ ]:
# prepare RMSD vs xtal structure
bb_rmss = list()
bb_rmsds = list()
bb_data = list()
sc_rmss = list()
sc_rmsds = list()
sc_data = list()

# backbone atoms
for u in unis:    # for all universes,
    for r in refs:  # for all reference structures,
        bb_rmss.append(rms.rmsd(u.select_atoms('backbone').positions, r.select_atoms('backbone').positions, center=True, superposition=True))  # append the rms analysis to the given position in rmss
        bb_rmsds.append(rms.RMSD(u, r, select='backbone', ref_frame=0))   # run RMSD analyses
for analysis in bb_rmsds:
    analysis.run()
    bb_data.append(pd.DataFrame(analysis.rmsd, columns=['Frame', 'Time (ps)', 'Backbone']))

# sidechain atoms
#sel = str(protein)      # normal protein
#sel = str()            # using selection algebra
sel = 'resnum 1:10 and not type H'            # resnumbers

for u in unis:    # for all universes,
    for r in refs:  # for all reference structures,
        sc_rmss.append(rms.rmsd(u.select_atoms(sel).positions, r.select_atoms(sel).positions, center=True, superposition=True))  # append the rms analysis to the given position in rmss
        sc_rmsds.append(rms.RMSD(u, r, select=sel, ref_frame=0))   # run RMSD analyses
for analysis in sc_rmsds:
    analysis.run()
    sc_data.append(pd.DataFrame(analysis.rmsd, columns=['Frame', 'Time (ps)', 'Residue']))

In [ ]:
bb_evs = [eigenvalues[0],eigenvalues[0],eigenvalues[0],eigenvalues[1],eigenvalues[1],eigenvalues[1]]
sc_evs = [eigenvalues[0],eigenvalues[0],eigenvalues[0],eigenvalues[1],eigenvalues[1],eigenvalues[1]]

In [ ]:
# prepare individual plots

for i in list(range(len(bb_rmss))):
    ax = bb_data[i].plot(x='Frame', y=['Backbone'], kind='line', figsize=(100,5), xlim=(-1,5001))
    ax = sc_data[i].plot(x='Frame', y=['Backbone'], kind='line', figsize=(100,5), xlim=(-1,5001))
    ax.set_ylabel('RMSD ($\AA$)')
    ax.set_xlabel('Time (ps)')
    #ax.xticks((1000*np.array(list(range(1,51)))))
    #ax.figure.savefig("rmsd_vs_acetone.pdf", bbox_inches="tight")
    ax2=ax.twinx()
    ax2.plot(time, bb_evs[i], label=labells, color='red')    # fixed for now!!! how do I make this oscillate properly??? make a 2D array of times/eigenvals/labels?
    #ax.figure.savefig("plt"+str(i)+".pdf", bbox_inches="tight")

In [ ]:
# TODO: 
# * compare rmsds of starting structures (xtal)
# * calculate RMSD of sidechains also?

In [ ]:
dlist = list()

for i in bb_data:
    dlist.append(i.values.tolist())

In [ ]:
#NECESSARY: parse the dataframes for precisely the data you need! test it out if you aren't sure...

#bb_data[1].plot(x='Time (ps)',)

# select the time and backbone w/in the dataframe,
df_times = bb_data[0]['Time (ps)']
df_bbrmsd = bb_data[0]['Backbone']

# then convert to lists!
t_list = df_times.values.tolist()
bbrmsd = df_bbrmsd.values.tolist()

In [ ]:
#pre-process/procure info from df
bb_rmsds = list()
for i in list(range(len(bb_rmss))):
    bb_rmsds.append(bb_data[i]['Backbone'])

# prepare large subplot
fig, axs = plt.subplots(6,1, sharex=True, sharey=True)
for i in list(range(len(axs))):
    #axs[i].plot(bb_data[i])
    #axs[i].plot(x='Frame', y=['Backbone'], kind='line')
    axs[i].plot(t_list, bb_rmsds[i])

    axs2=axs[i].twinx()
    axs2.plot(t_list, bb_evs[i], label=fig_labl[i], color='red')
    #axs.xlim(-1,5001)
    axs[i].set_xlim([-1,50001])      # set x limit for figure    
    #axs[i].legend(loc=3)
    axs[i].set_ylabel('RMSD ($\AA$)')
    axs[i].set_xlabel('Time (ps)')
    axs[i].set_title(fig_labl[i], loc='left')       # label with the specified labels (automate later!); move the title to the LHS of the page
    #axs[i].xticks(list(range(0,1000,50000)))

fig.set_size_inches(100,15)
fig.suptitle('Coincidence of RMSD relative to xtal structure (red) with conformational eigenvalue (blue) versus time (ps). convention=STRUCT_T_SOLV_RMSDvs')
fig.savefig("subplots.pdf", bbox_inches="tight")